In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
%pwd

'd:\\ML projects\\Wine prediction\\Wine_prediction'

In [25]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str

In [26]:
from Wine_prediction.constants import *
from Wine_prediction.utils.common import read_yaml, create_directories, save_json

In [27]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.RandomForestClassifier
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name
           
        )

        return model_evaluation_config

In [32]:
import os
import pandas as pd
from sklearn.metrics import accuracy_score as acc_score, confusion_matrix as conf_matrix, classification_report as class_report

from urllib.parse import urlparse
import numpy as np
import joblib
import json

In [36]:
import pandas as pd
import joblib
import json

class ModelEvaluation:
    def __init__(self, model_evaluation_config):
        self.config = model_evaluation_config

    def eval_metrics(self, actual, pred):
        # Define or import functions like acc_score, conf_matrix, class_report here
        accuracy = acc_score(actual, pred)
        confusion = conf_matrix(actual, pred)
        classification = class_report(actual, pred)
        return accuracy, confusion, classification

    def save_results(self):
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]
        
        predicted_qualities = model.predict(test_x)

        accuracy_score, confusion_matrix, classification_report = self.eval_metrics(test_y, predicted_qualities)
        
        # Convert NumPy arrays to serializable lists
        confusion_matrix = confusion_matrix.tolist()
        
        # Ensure classification_report is in JSON format before using json.loads()
        # Example assuming classification_report is a dictionary that needs to be converted to JSON
        classification_report = json.dumps(classification_report)

        # Saving metrics as local
        scores = {"accuracy_score": accuracy_score, "confusion_matrix": confusion_matrix, "classification_report": classification_report}
        save_json(path=Path(self.config.metric_file_name), data=scores)


In [37]:
config_manager = ConfigurationManager()
model_evaluation_config = config_manager.get_model_evaluation_config()
model_evaluation = ModelEvaluation(model_evaluation_config)
model_evaluation.save_results()

[2024-01-10 18:55:13,396: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-10 18:55:13,404: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-10 18:55:13,421: INFO: common: yaml file: schema.yaml loaded successfully]
[2024-01-10 18:55:13,430: INFO: common: created directory at: artifacts]
[2024-01-10 18:55:13,437: INFO: common: created directory at: artifacts/model_evaluation]


[2024-01-10 18:55:14,424: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


d:\ML projects\Wine prediction\Wine_prediction\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ML projects\Wine prediction\Wine_prediction\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
d:\ML projects\Wine prediction\Wine_prediction\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_st